<a href="https://colab.research.google.com/github/fora22/Data_Science/blob/main/Ensemble_%26_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 그래프 한글 폰트 설정
!apt-get update -qq
!apt-get install fonts-nanum* -qq
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname = path, size = 10).get_name()
print(font_name)
plt.rc('font', family=font_name)  
fm._rebuild()

NanumBarunGothic


# 투표 기반 분류기

정확도가 80%인 분류기 여러 개를 훈련시켰다고 가정한다. 로지스틱 회귀 분류기, SVM 분류기, 랜덤 포레스트 분류기, K-최근접 이웃 분류기 등등

![image](https://user-images.githubusercontent.com/48875566/130360636-1a19813e-ffee-4d80-a647-ff3c20551edd.png)

[그림 출처](https://hoony-gunputer.tistory.com/entry/%ED%95%B8%EC%A6%88%EC%98%A8-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-7%EA%B0%95-%EC%95%99%EC%83%81%EB%B8%94-%ED%95%99%EC%8A%B5%EA%B3%BC-%EB%9E%9C%EB%8D%A4-%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8ensemble-RandomForest?category=742600)

더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것이다. 이렇게 다수결 투표로 정해지는 분류기를 직접 투표(hard voting) 분류기라고 한다.

이 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높을 경우가 많다. 사실 각 분류기가 **약한 학습기**일지라도 충분하게 많고 다양하다면 앙상블은 **강한 학습기**가 될 수 있다.



In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [4]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [5]:
voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)

In [6]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [7]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score

In [8]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 1.0
SVC 0.95
VotingClassifier 0.95


모든 분류기가 클래스의 확률을 예측할 수 있으면, 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 **간접 투표(soft voting)**라고 한다. 이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다.

이 방식을 사용하기 위해서는 `voting = 'hard'`를 `voting = 'soft'`로 바꾸고 보든 분류기가 클래스의 확률을 추정할 수 있으면 된다. `SVC`는 기본값에서는 클래스 확률을 제공하지 않으므로 `probability` 매개 변수를 `True`로 지정해야 한다. 대신 이렇게 하면 교차 검증으로 인해 훈련 속도가 느려진다.

In [9]:
# 간접 투표 실행
log_clf_soft = LogisticRegression()
rnd_clf_soft = RandomForestClassifier()
svm_clf_soft = SVC(probability=True)

In [10]:
voting_clf_soft = VotingClassifier(
    estimators = [('lr', log_clf_soft), ('rf', rnd_clf_soft), ('svc', svm_clf_soft)],
    voting = 'soft'
)

In [11]:
for clf_soft in (log_clf_soft, rnd_clf_soft, svm_clf_soft, voting_clf_soft):
    clf_soft.fit(X_train, y_train)
    y_pred = clf_soft.predict(X_test)
    print(clf_soft.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 1.0
SVC 0.95
VotingClassifier 1.0
